# Importing the Basic Libraries

In [ ]:
import numpy as np 
import pandas as pd 

# Importing the ImageDataGenerator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

ImageDataGenerator is used to load the images from the respective Directory with the mentioned batch size and Image size, also several augments of the image can also be made with the ImageDataGenerator

<b>Here the given list of augments has been made to improve the learning.</b>

1)Width Shift (Shifting the Image Horizontally)

2)Height Shift (Shifting the Image Vertically)

3)Rescale (Converting the pixel values from 0 - 255 to 0 - 1)

4)Shear (Shifting one part of a Image) <a href="https://docs.gimp.org/2.10/en/gimp-tool-shear.html#:~:text=Shear%20tool%20is%20used%20to,A%20rectangle%20becomes%20a%20diamond.">Please refer this link for shear Example</a>

5)Zoom

In [ ]:
image_gen = ImageDataGenerator(
                               width_shift_range=0.1, 
                               height_shift_range=0.1, 
                               rescale=1/255, 
                               shear_range=0.2, 
                               zoom_range=0.2, 
                               fill_mode='nearest'
                              )

Here all the Images have been resized to 350 x 350 in black and white mode and used <br>

Batch size of 16 is used <br>

The images have been loaded from the Directory(recognizance/train) in B/W mode with ImageDataGenerator <br>

The ImageDataGenerator Now reads 16 images in B/W mode each time

In [ ]:
image_shape = (350,350,1)
batch_size = 16
train_image_gen = image_gen.flow_from_directory('../input/recognizance/train',color_mode='grayscale',
                                               target_size=image_shape[:2],
                                               batch_size=batch_size,
                                               class_mode='binary',seed=42)

# Modeling

The CNN model is implemented using the Keras Library. <br>

Loss Function used - Binary Crossentropy <br>
Optimizer - Adam <br>
Epochs Used - 30

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense, Conv2D, MaxPool2D

In [ ]:
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(350,350,1), activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))


model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.fit_generator(generator=train_image_gen,epochs=20)

In [ ]:
model.fit_generator(generator=train_image_gen,epochs=5)

In [ ]:
model.fit_generator(generator=train_image_gen,epochs=5)

# Predicting the Test Data

All the filenames inside the test folder are read and stored in a array

In [ ]:
import os
files = []

for dirname, _, filenames in os.walk('/kaggle/input/recognizance/test/'):
    for filename in filenames:
        files.append(filename)

Then each corresponding image is loaded using Keras.image.load_img in B/W mode and with dimension of 350 <br>

Then the class of the Image(Positive/negative) is predicted and the value is appended in a Array

In [ ]:
pred = []
for i in files:
    img = image.load_img('../input/recognizance/test/'+i, target_size=(350,350,1),color_mode = "grayscale")
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img/255
    pred.append(model.predict_classes(img)[0][0])

Then the File names with their classes are made into a Dataframe for submission

In [ ]:
sub = pd.DataFrame(pred,files).reset_index()
sub.columns = ['image','label']
sub.to_csv('sub4.csv',index=False)

In [ ]:
!gzip sub4.csv

In [ ]:
model.save("model.h5")

In [ ]:
model.save_weights("modelweights.h5")

In [ ]:
from IPython.display import FileLink
FileLink(r'modelweights.h5')